In [3]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [1]:
print("Hello")

Hello


In [2]:
from keras.datasets import mnist
from keras.utils import np_utils
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os

# Training Parameters
batch_size = 64
epochs = 10

# loads the MNIST dataset
(x_train, y_train), (x_test, y_test)  = mnist.load_data()

# Lets store the number of rows and columns
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

# Getting our date in the right 'shape' needed for Keras
# We need to add a 4th dimenion to our date thereby changing our
# Our original image shape of (60000,28,28) to (60000,28,28,1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# store the shape of a single image 
input_shape = (img_rows, img_cols, 1)

# change our image type to float32 data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Now we one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Let's count the number columns in our hot encoded matrix 
print ("Number of Classes: " + str(y_test.shape[1]))

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

# create model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adadelta(),
              metrics = ['accuracy'])

print(model.summary())

                     
checkpoint = ModelCheckpoint("MNIST_Checkpoint.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)
callbacks = [checkpoint]


Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Number of Classes: 10
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________

In [3]:
history = model.fit(x_train, y_train,
          batch_size = batch_size,
          epochs = epochs,
          verbose = 1,
          callbacks = callbacks,
          validation_data = (x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 11s 182us/step - loss: 0.2171 - accuracy: 0.9335 - val_loss: 0.0507 - val_accuracy: 0.9823

Epoch 00001: val_loss improved from inf to 0.05068, saving model to MNIST_Checkpoint.h5
Epoch 2/10
60000/60000 [==============================] - 8s 140us/step - loss: 0.0820 - accuracy: 0.9757 - val_loss: 0.0398 - val_accuracy: 0.9865

Epoch 00002: val_loss improved from 0.05068 to 0.03976, saving model to MNIST_Checkpoint.h5
Epoch 3/10
60000/60000 [==============================] - 9s 143us/step - loss: 0.0631 - accuracy: 0.9813 - val_loss: 0.0344 - val_accuracy: 0.9884

Epoch 00003: val_loss improved from 0.03976 to 0.03444, saving model to MNIST_Checkpoint.h5
Epoch 4/10
60000/60000 [==============================] - 9s 154us/step - loss: 0.0529 - accuracy: 0.9841 - val_loss: 0.0332 - val_accuracy: 0.9891

Epoch 00004: val_loss improved from 0.03444 to 0.03316, saving model to MNIST_Che

### Adding Multiple Call Backs & Early Stopping

We can use other call back methods to monitor our training process such as **Early Stopping**. Checkout the Keras documentation for more:
- https://keras.io/callbacks/

In [5]:
from keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor = 'val_loss', # value being monitored for improvement
                          min_delta = 0, #Abs value and is the min change required before we stop
                          patience = 3, #Number of epochs we wait before stopping 
                          verbose = 1,
                          restore_best_weights = True) #keeps the best weigths once stopped

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

### We can attempt to run again to see if it worked!

In [8]:
history = model.fit(x_train, y_train,
          batch_size=64,
          epochs=30,
          verbose=1,
          callbacks = callbacks,
          validation_data=(x_test, y_test))


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/30
60000/60000 [==============================] - 10s 162us/step - loss: 0.0281 - accuracy: 0.9915 - val_loss: 0.0297 - val_accuracy: 0.9913

Epoch 00001: val_loss did not improve from 0.02514
Epoch 2/30
60000/60000 [==============================] - 9s 156us/step - loss: 0.0268 - accuracy: 0.9922 - val_loss: 0.0300 - val_accuracy: 0.9925

Epoch 00002: val_loss did not improve from 0.02514
Epoch 3/30
60000/60000 [==============================] - 10s 160us/step - loss: 0.0264 - accuracy: 0.9922 - val_loss: 0.0257 - val_accuracy: 0.9926

Epoch 00003: val_loss did not improve from 0.02514
Epoch 4/30
60000/60000 [==============================] - 9s 158us/step - loss: 0.0264 - accuracy: 0.9922 - val_loss: 0.0267 - val_accuracy: 0.9922

Epoch 00004: val_loss did not improve from 0.02514
Epoch 5/30
60000/60000 [==============================] - 9s 149us/step - loss: 0.0257 - accuracy: 0.9919 - val_loss: 0.0254 - val_accuracy: 0.9922


### Another useful callback is Reducing our learning Rate on Plateau

We can avoid having our oscillate around the global minimum by attempting to reduce the Learn Rate by a certain fact. If no improvement is seen in our monitored metric (val_loss typically), we wait a certain number of epochs (patience) then this callback reduces the learning rate by a factor

In [9]:
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)